In [1]:
import mmcv
import mmengine
import torch
import matplotlib.pyplot as plt
from mmdet.apis import init_detector, inference_detector
from tabulate import tabulate
import matplotlib.patches as mpatches
import numpy as np
from PIL import Image
import cv2
from mmseg.apis import inference_model, init_model
import mmdet
import mmseg
import mmyolo
import copy
print('mmcv', mmcv.__version__)
print('mmengine', mmengine.__version__)
print('mmdet', mmdet.__version__)
print('mmseg', mmseg.__version__)
print('mmyolo', mmyolo.__version__)


mmcv 2.0.0rc4
mmengine 0.6.0
mmdet 3.0.0rc6
mmseg 1.0.0rc6
mmyolo 0.5.0


### Model Initialization

#### Init Segmentor

In [2]:
data_root = '../mmsegmentation/data/IonoSeg/'
img_dir = 'rgbimg'
ann_dir = 'rgbmask'
classes = ('Background', 'E', 'Es-l', 'Es-c', 'F1', 'F2', 'Spread-F')
palette = [[255, 255, 255], [250, 165, 30], [120, 69, 125], [53, 125, 34], 
           [0, 11, 123], [130, 20, 12], [120, 121, 80]] # F1 [78, 210, 240]
patches = [mpatches.Patch(color=np.array(palette[i]) / 255., label=classes[i]) for i in range(7)]

seg_config_file = '../mmsegmentation/configs/_se4ionogram/pspnet_r50_ionogram_mmseg1.py'
seg_checkpoint_file = '../mmsegmentation/work_dirs/se4ionogram/pspnet_r50_ionogram_iou_3922_acc_9153.pth'

seg_model = init_model(seg_config_file, seg_checkpoint_file, device='cuda:0')

/home/ubuntu/mmsegmentation/mmseg/models/builder.py:36: UserWarning: ``build_loss`` would be deprecated soon, please use ``mmseg.registry.MODELS.build()`` 
  warnings.warn('``build_loss`` would be deprecated soon, please use '
/home/ubuntu/mmsegmentation/mmseg/models/losses/cross_entropy_loss.py:235: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  warnings.warn(


Loads checkpoint by local backend from path: ../mmsegmentation/work_dirs/se4ionogram/pspnet_r50_ionogram_iou_3922_acc_9153.pth


In [3]:
from mmseg.registry import DATASETS
from mmseg.datasets import BaseSegDataset


@DATASETS.register_module()
class IonogramSegmentationDataset(BaseSegDataset):
  METAINFO = dict(classes = classes, palette = palette)
  def __init__(self, **kwargs):
    super().__init__(img_suffix='.png', seg_map_suffix='.png', **kwargs)

In [4]:
def single_load(model, img):
    segs = inference_model(model, img)
    segmap = segs.pred_sem_seg.data.cpu().squeeze(0).numpy()
    return segmap

def post_seg(segmap, img):
    blurred = cv2.GaussianBlur(img, (3, 3), 0)
    edge = cv2.Canny(blurred, 50, 150)
    edge = edge / 255
    segmap = segmap * edge   # Hadamard product
    return segmap

def interpret_numpy(segmap):
    labels = []
    parameters = []
    for class_id in range(1, 7):
        indices = np.where(segmap == class_id)
        if len(indices[0]) != 0:
            parameters.append([np.max(indices[1]), np.max(indices[0])])
            labels.append(class_id - 1)
    # # fmin
    # indices = np.nonzero(segmap)
    # if len(indices[0]) != 0:
    #     fmin = np.min(indices[1])
    return [torch.tensor(parameters), labels]

#### Init Detector

In [5]:
val_dir = './Iono4311/val_images'
test_dir = './Iono4311/test_images'
ann_val = './Iono4311/annotations/val.json'
ann_test = './Iono4311/annotations/test.json'

# Choose to use a config and initialize the detector
det_config_file = './configs/custom_dataset/rtmdet/rtmdet_tiny_fast_1xb32-100e_ionogram.py'
# Setup a checkpoint file to load
det_checkpoint_file = './work_dirs/rtmdet_tiny_100e/best_coco/bbox_mAP_epoch_100_0.589.pth'

# build the model from a config file and a checkpoint file
det_model = init_detector(det_config_file, det_checkpoint_file)
det_model.cfg['model']['test_cfg']['nms']['iou_threshold'] = 0.65
print(det_model.cfg['model']['test_cfg'])

annotation = mmengine.load(ann_test)
name_list = []
for image in annotation['images']:
    name_list.append(image['file_name'][:-4])
print(f'{len(name_list)} images in the test set')
# 建立 name2index 索引
name2annid = {}
for image in annotation['images']:
    ann_id = []
    for i, ann in enumerate(annotation['annotations']):
        if ann['image_id'] == image['id']:
            ann_id.append(i)
    name2annid[image['file_name']] = ann_id

img_names = []
for image in annotation['images']:
    img_names.append(image['file_name'])

Loads checkpoint by local backend from path: ./work_dirs/rtmdet_tiny_100e/best_coco/bbox_mAP_epoch_100_0.589.pth
{'multi_label': True, 'nms_pre': 30000, 'score_thr': 0.001, 'nms': {'type': 'nms', 'iou_threshold': 0.65}, 'max_per_img': 100}
646 images in the test set


In [6]:
def get_detection_pred(model, img):
    pred = inference_detector(model, img)
    pred_instances = pred.pred_instances
    pred_instances = pred_instances[pred_instances.scores > 0.3]

    # 每一类只保留一个置信度最高的 bbox
    max_idx = []
    for i in range(torch.max(pred_instances.labels) + 1):
        idx = torch.where(pred_instances.labels == i)[0]
        if len(idx) > 0:
            max_idx.append(idx[torch.argmax(pred_instances.scores[idx])])
    max_idx = torch.tensor(max_idx)
    scores = pred_instances.scores[max_idx].tolist()
    labels = pred_instances.labels[max_idx].tolist()
    bboxes = pred_instances.bboxes[max_idx].cpu()
    params = bboxes[:, 2:4] # [height, frequency]
    return bboxes, params, labels, scores

def get_segmentation_pred(model, img):
    segmap = single_load(model, img)
    segmap = post_seg(segmap, img)
    parameters, labels = interpret_numpy(segmap)
    return [segmap, parameters, labels]

def interprete_parameters(boundaries, labels):
    # [hE, foE, hEs, foEs, hF1, foF1, hF2, foF2] = [-1] * 8
    params = [-1] * 8
    # labels: parameter_id      labels begin with 0
    boundary2param = {
        0: 0,       # E
        1: 2, 2: 2, # Es
        3: 4,       # F1
        4: 6, 5: 6  # F2, Spread-F
    }
    for i in range(len(labels)):
        params[boundary2param[labels[i]]: boundary2param[labels[i]] + 2] = boundaries[i].tolist()
    return params

def get_gt(file_name):
    # bbox [x1, y1, x2, y2] 
    # x1 refers to the distance between the upperleft point and left boundary,
    # y1 refers to the distance between the upperleft point and the top boundary.
    labels = [annotation['annotations'][i]['category_id'] - 1 for i in name2annid[file_name]]   # labels begin with 0
    bboxes = [annotation['annotations'][i]['bbox'] for i in name2annid[file_name]]
    bboxes = torch.tensor(bboxes)
    parameters = [[box[0] + box[2], box[1] + box[3]] for box in bboxes]
    parameters = torch.tensor(parameters)
    bboxes[:, 2:4] = parameters
    mask = Image.open(f'{data_root}/{ann_dir}/{file_name}')
    return [mask, bboxes, parameters, labels]

### Plot

In [7]:
colors = [[0.98, 0.647, 0.118], [0.471, 0.271, 0.49], [0.208, 0.49, 0.133], [0.307, 0.823, 0.941], [0.51, 0.078, 0.047], [0.471, 0.475, 0.314]]
def show_bboxes(axes, bboxes, catogories, labels=None):

    def make_list(obj, default_values=None):
        if obj is None:
            obj = default_values
        elif not isinstance(obj, (list, tuple)):
            obj = [obj]
        return obj

    labels = make_list(labels)
    for i, bbox in enumerate(bboxes):
        color = colors[catogories[i]]
        rect = plt.Rectangle(
            xy=(bbox[0], bbox[1]), width=bbox[2]-bbox[0],
            height=bbox[3]-bbox[1], fill=False, edgecolor=color, linewidth=1.5)
        axes.add_patch(rect)
        if labels and len(labels) > i:
            text_color = 'w'
            axes.text(rect.xy[0], rect.xy[1] - 6, labels[i],
                      va='center', ha='center', fontsize=9, color=text_color,
                      bbox=dict(facecolor=color, lw=0, pad=0.5))

def set_ticks(ax):
    ax.set_xticks([0, 50, 100, 150, 200, 250, 300, 350, 400], [1.0, 3.5, 6.0, 8.5, 11.0, 13.5, 16.0, 18.5, 21.0])
    ax.set_yticks([0, 40, 80, 120, 160, 200, 240, 280, 320, 360], [990, 890, 790, 690, 590, 490, 390, 290, 190, 90])
    ax.set_xlabel('Frequency (MHz)')
    ax.set_ylabel('Virtual Height (km)')

def plot_layer_parameters(ax, layer_name, h, f, color='r', h_offset=0, f_offset=0):
    if h > 0:
        ax.axvline(f, color=color, linestyle=":", alpha=0.8)    # foX
        ax.text(h_offset - 10, -3, f'fo{layer_name}', color=color)
        ax.axhline(h, color=color, linestyle="--", alpha=0.8)   # h'X
        ax.text(401, f_offset + 2, f'h\'{layer_name}', color=color)

# params = [hE, foE, hEs, foEs, hF1, foF1, hF2, foF2]
boundary2param = {
    'E': {'offset': -6, 'param_index':[1, 0], 'color': [colors[0], colors[0]]},
    'Es': {'offset': 0, 'param_index':[3, 2], 'color': [colors[2], colors[1]]},
    'F1': {'offset': 0, 'param_index':[5, 4], 'color': [colors[3], colors[3]]},
    'F2': {'offset': 0, 'param_index':[7, 6], 'color': [colors[4], colors[5]]}
}

def params2txtpos(params, width):
    sorted_params = sorted(params)   # 递增
    text_pos = copy.deepcopy(sorted_params)
    l = len(params)
    deviation = [sorted_params[i + 1] - sorted_params[i] for i in range(0, l-1)]
    for i in range(len(deviation)):
        if deviation[i] < width and deviation[i] != 0:
            text_pos[i] -= (width-deviation[i]) / 2 
            text_pos[i+1] += (width-deviation[i]) / 2 
    mapping = dict(zip(sorted_params, text_pos))
    resorted_lst = [mapping[x] for x in params]

    return resorted_lst

def plot_parameters(ax, params, labels):
    txtpos_f = params2txtpos(params[1:8:2], width=14)
    txtpos_h = params2txtpos(params[0:8:2], width=26)
    for key in boundary2param.keys():
        if key == 'Es' and 2 in labels:
            color = boundary2param[key]['color'][0]
        elif key == 'F2' and 4 in labels: # Es-c or F2
            color = boundary2param[key]['color'][0]
        else:
            color = boundary2param[key]['color'][1]
        plot_layer_parameters(ax, key, params[boundary2param[key]['param_index'][0]], params[boundary2param[key]['param_index'][1]], color=color, 
                              h_offset=txtpos_h[boundary2param[key]['param_index'][1] // 2], f_offset=txtpos_f[boundary2param[key]['param_index'][1] // 2])

def show_det_result(file_name, bboxes, boundaries_pred, labels, figsize=(18, 6)):
    _, axs = plt.subplots(1, 3, figsize=figsize)
    img = mmcv.imread(test_dir + '/' + file_name)
    [axs[i].imshow(255 - img) for i in range(3)]
    axs[0].set_title('Input Ionogram', y=1.018)
    axs[1].set_title('Pred Bboxes', y=1.018)
    axs[2].set_title('Ground Truth', y=1.018)

    if isinstance(bboxes, torch.Tensor):    # x1, y1, x2, y2
        bboxes = bboxes.cpu()
    mask, bboxes_gt, boundaries_gt, labels_gt = get_gt(file_name)
    params_pred = interprete_parameters(boundaries_pred, labels)
    params_gt = interprete_parameters(boundaries_gt, labels_gt) # [hE, foE, hEs, foEs, hF1, foF1, hF2, foF2]
    for ax in axs:
        set_ticks(ax)
    plot_parameters(axs[1], params_pred, labels)
    plot_parameters(axs[2], params_gt, labels_gt)
    show_bboxes(axs[1], bboxes, catogories=labels, labels=[det_model.dataset_meta['classes'][labels[i]] for i in range(len(labels))])
    show_bboxes(axs[2], bboxes_gt, catogories=labels_gt, labels=[det_model.dataset_meta['classes'][labels_gt[i]] for i in range(len(labels_gt))])

def show_seg_result(file_name, segmap, boundaries_pred, labels, figsize=(18, 6)):
    _, axs = plt.subplots(1, 3, figsize=figsize)
    img = mmcv.imread(test_dir + '/' + file_name)
    mask, _, boundaries_gt, labels_gt = get_gt(file_name)
    axs[0].imshow(img)
    axs[2].imshow(mask)
    seg_res = Image.fromarray(np.uint8(segmap)).convert('P')
    seg_res.putpalette(np.array(palette, dtype=np.uint8))
    axs[1].imshow(np.array(seg_res.convert('RGB')))
    axs[0].set_title('Input Ionogram', y=1.018)
    axs[1].set_title('Seg Map', y=1.018)
    axs[2].set_title('Ground Truth', y=1.018)
    for ax in axs: 
        set_ticks(ax)
    params_pred = interprete_parameters(boundaries_pred, labels)
    params_gt = interprete_parameters(boundaries_gt, labels_gt) # [hE, foE, hEs, foEs, hF1, foF1, hF2, foF2]
    plot_parameters(axs[1], params_pred, labels)
    plot_parameters(axs[2], params_gt, labels_gt)

### Custom Evaluate

In [8]:
def boundaries_within_total_threshold(boundaries_pred, boundaries_gt, threshold=10):
    return torch.sum(torch.abs(boundaries_pred - boundaries_gt)) <= threshold

def all_boundaries_within_threshold(boundaries_pred, boundaries_gt, threshold=4):
    return torch.all(torch.abs(boundaries_pred - boundaries_gt)<= threshold)

def calculate_metrics(TP, FP, num_gts):
    precisions, recalls = [], []
    for i in range(6):
        if num_gts[i] == 0:
            precisions.append(-1)
            recalls.append(-1)
        else:
            precisions.append(TP[i] / (TP[i] + FP[i]))
            recalls.append(TP[i] / num_gts[i])

    # Calculate Metrics
    mean_precision = sum(precisions) / 6
    mean_recall = sum(recalls) / 6
    F1_score = 2 * mean_precision * mean_recall / (mean_precision + mean_recall)
    overall_precision = sum(TP) / (sum(TP) + sum(FP))
    overall_recall = sum(TP) / sum(num_gts)
    Layer_names = list(det_model.dataset_meta['classes'])
    Layer_names.extend(['Mean', 'Overall'])
    precisions.extend([mean_precision, overall_precision])
    recalls.extend([mean_recall, overall_recall])
    formatted_precisions = ['{:.4f}'.format(precision) for precision in precisions]
    formatted_recalls = ['{:.4f}'.format(recall) for recall in recalls]
    result_table = list(zip(Layer_names, formatted_precisions, formatted_recalls))
    print(tabulate(result_table, headers=['Layer', 'Precision', 'Recall']))
    print(f'F1_score            {F1_score:.4f}')

In [9]:
TP, FP = [0] * 6, [0] * 6
num_gts = [0] * 6
cnt = 0
sample_interval = 1
det_model = init_detector(det_config_file, det_checkpoint_file)
# seg_model = init_model(seg_config_file, seg_checkpoint_file, device='cuda:0')

# for image in annotation['images']:
while cnt < len(annotation['images']):
    img = mmcv.imread(test_dir + '/' + img_names[cnt])
    # Get detection predictions
    bboxes, boundaries_pred, labels, scores = get_detection_pred(det_model, img)
    # Get segmentation predictions
    # segmap, boundaries_pred, labels = get_segmentation_pred(seg_model, img)

    # Get the ground truth in lists
    mask, bboxes_gt, boundaries_gt, labels_gt = get_gt(img_names[cnt])
    
    # Interprete Prameters
    params_pred = interprete_parameters(boundaries_pred, labels)
    params_gt = interprete_parameters(boundaries_gt, labels_gt)
    # print(cnt)
    # show_det_result(img_names[cnt], bboxes, boundaries_pred, labels)
    # show_seg_result(img_names[cnt], segmap, boundaries_pred, labels)
    
    for label in labels_gt:
        num_gts[label] += 1
    for i in range(len(labels)):
        if labels[i] in labels_gt and all_boundaries_within_threshold(boundaries_pred[i], boundaries_gt[labels_gt.index(labels[i])]):
            TP[labels[i]] += 1
        else:
            FP[labels[i]] += 1
    cnt += sample_interval
    if cnt > len(annotation['images']):
        break

calculate_metrics(TP, FP, num_gts)

Loads checkpoint by local backend from path: ./work_dirs/rtmdet_tiny_100e/best_coco/bbox_mAP_epoch_100_0.589.pth


/home/ubuntu/.conda/envs/mmlab/lib/python3.8/site-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1659484810403/work/aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Layer       Precision    Recall
--------  -----------  --------
E              0.8845    0.9147
Es-l           0.7553    0.5772
Es-c           0.791     0.8346
F1             0.8867    0.9226
F2             0.9634    0.9711
Spread-F       0.6176    0.9545
Mean           0.8164    0.8625
Overall        0.8955    0.9057
F1_score            0.8388


In [10]:
'''
0: E, Es-c, F1, F2
70: Es-l, F2
76: Spread-F

先验知识:
单独存在的 Es-c -> Es-l
Spread-F + F2 -> 取置信度高的
E + Es-l -> E + Es-c 
'''

'\n0: E, Es-c, F1, F2\n70: Es-l, F2\n76: Spread-F\n\n先验知识:\n单独存在的 Es-c -> Es-l\nSpread-F + F2 -> 取置信度高的\nE + Es-l -> E + Es-c \n'

**seg** 
Layer       Precision    Recall
--------  -----------  --------
E              0.7621    0.8089
Es-l           0.4805    0.6016
Es-c           0.5924    0.7323
F1             0.7941    0.8182
F2             0.9665    0.9711
Spread-F       0.6522    0.6818
Mean           0.7079    0.769
Overall        0.8034    0.8532
F1_score            0.7372

**det**
Layer       Precision    Recall
--------  -----------  --------
E              0.8845    0.9147
Es-l           0.7553    0.5772
Es-c           0.791     0.8346
F1             0.8867    0.9226
F2             0.9634    0.9711
Spread-F       0.6176    0.9545
Mean           0.8164    0.8625
Overall        0.8955    0.9057
F1_score            0.8388

### COCO Evaluation

In [11]:
# results_COCO = []
# for image in annotation['images']:
#     img = mmcv.imread(test_dir + '/' + image['file_name'])
#     pred = inference_detector(model, img)
#     pred_instances = pred.pred_instances
#     pred_instances = pred_instances[pred_instances.scores > 0.1]
#     bboxs = pred_instances.bboxes
#     bboxs[:, 2], bboxs[:, 3] =  bboxs[:, 2] - bboxs[:, 0], bboxs[:, 3] - bboxs[:, 1]
#     bboxs = bboxs.tolist()
#     scores = pred_instances.scores.tolist()
#     labels = pred_instances.labels.tolist()
#     for i in range(len(labels)):
#         pred_dict = {
#             "image_id": image['id'],
#             "category_id": labels[i] + 1,
#             "bbox": bboxs[i],   # [x, y, width, height]
#             "score": scores[i]
#         }
#         results_COCO.append(pred_dict)
# mmengine.dump(results_COCO, './results_COCO.json')
# len(results_COCO)

In [12]:
# from pycocotools.coco import COCO
# from pycocotools.cocoeval import COCOeval

# # 初始化 COCO API
# annFile = ann_test  # COCO 数据集的标注文件路径
# cocoGt = COCO(annFile)

# # 载入检测结果
# # resFile = './results_COCO.json'  # 检测结果文件路径
# cocoDt = cocoGt.loadRes(results_COCO)

# # 创建评估器
# cocoEval = COCOeval(cocoGt, cocoDt, 'bbox')

# # 运行评估并获取结果
# cocoEval.evaluate()
# cocoEval.accumulate()
# cocoEval.summarize()

# # 输出 precisions 和 recall
# print('precisions: ', cocoEval.stats[0])
# print('recall: ', cocoEval.stats[8])

# for catId in cocoGt.getCatIds():
#     coco_eval = COCOeval(cocoGt, cocoDt, 'bbox')
#     coco_eval.params.catIds = [catId]
#     coco_eval.evaluate()
#     coco_eval.accumulate()
#     coco_eval.summarize()

### Draft

In [13]:
# from mmdet.registry import VISUALIZERS
# visualizer = VISUALIZERS.build(det_model.cfg.visualizer)
# visualizer.dataset_meta = det_model.dataset_meta

# image = mmcv.imread(test_dir + '/' +  annotation['images'][0]['file_name'])
# print('id ', annotation['images'][0]['id'], annotation['images'][0]['file_name'])
# result = inference_detector(det_model, image)
# print(result)

# # show the results
# visualizer.add_datasample(
#     'result',
#     image,
#     data_sample=result,
#     draw_gt = None,
#     wait_time=0,
# )
# visualizer.show()

In [14]:
import numpy as np
def calculate_iou(box1, box2):
    # 计算两个矩形框的交集和并集
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    intersection = max(0, x2 - x1) * max(0, y2 - y1)
    union = (box1[2] - box1[0]) * (box1[3] - box1[1]) + (box2[2] - box2[0]) * (box2[3] - box2[1]) - intersection
    
    # 计算 IoU 值
    iou = intersection / union if union > 0 else 0
    return iou

detections = np.array([
    [50, 50, 100, 100, 0.9],
    [150, 150, 200, 200, 0.8],
    [75, 75, 125, 125, 0.7],
    [130, 130, 180, 180, 0.6],
    [90, 90, 140, 140, 0.5],
    [10, 10, 60, 60, 0.4],
    [200, 200, 250, 250, 0.3],
    [170, 170, 220, 220, 0.2],
    [110, 110, 160, 160, 0.1],
    [20, 20, 70, 70, 0.0]
])

gt_boxes = np.array([
    [60, 60, 110, 110],
    [155, 155, 205, 205],
    [80, 80, 130, 130],
    [130, 130, 180, 180],
    [100, 100, 150, 150]
])

# 按照置信度从高到低排序
sorted_indices = np.argsort(-detections[:, 4])
sorted_detections = detections[sorted_indices]

# 计算每个检测结果与所有 ground truth 的 IoU 值
ious = np.zeros((len(sorted_detections), len(gt_boxes)))
for i, detection in enumerate(sorted_detections):
    for j, gt_box in enumerate(gt_boxes):
        iou = calculate_iou(detection[:4], gt_box)
        ious[i, j] = iou

# 对每个检测结果，选择与其 IoU 值最大的 ground truth 进行匹配
max_ious = ious.max(axis=1)
matched_indices = ious.argmax(axis=1)

# 根据 IoU 阈值选择 TP 和 FP
iou_threshold = 0.5
tp_flags = max_ious >= iou_threshold
fp_flags = max_ious < iou_threshold
fp_flags |= matched_indices == -1

# 将 TP 和 FP 的标记添加到检测结果数组中
sorted_detections = np.hstack((sorted_detections, tp_flags[:, np.newaxis], fp_flags[:, np.newaxis]))

# 计算 Precision 和 Recall
tp_cumsum = np.cumsum(tp_flags)
fp_cumsum = np.cumsum(fp_flags)
recall = tp_cumsum / len(gt_boxes)
precision = tp_cumsum / (tp_cumsum + fp_cumsum)

# 计算 AP50 值
ap50 = 0.
for t in np.arange(0., 1.1, 0.1):
    mask = recall >= t
    if mask.any():
        ap50 += np.max(precision[mask]) / 11.

# 输出结果
print('AP50: {:.4f}'.format(ap50))
print('Recall: {:.4f}'.format(recall[-1]))

AP50: 0.4091
Recall: 0.6000


In [15]:
#  Average Precision  (precisions) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.588
#  Average Precision  (precisions) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.874
#  Average Precision  (precisions) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.618
#  Average Precision  (precisions) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.543
#  Average Precision  (precisions) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.715
#  Average Precision  (precisions) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.894
#  Average Recall     (recalls) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.632
#  Average Recall     (recalls) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.670
#  Average Recall     (recalls) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.670
#  Average Recall     (recalls) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.637
#  Average Recall     (recalls) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.760
#  Average Recall     (recalls) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.925

# bbox_mAP_copypaste: 0.589 0.876 0.621 0.544 0.723 0.897